In [1]:
! pip3 install s3fs

  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec None
error: uninstall-no-record-file

× Cannot uninstall fsspec None
╰─> The package's contents are unknown: no RECORD file was found for fsspec.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps fsspec==2023.6.0


# import libraries

In [2]:
import boto3
import pandas as pd 

from sagemaker import get_execution_role
import sagemaker
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Split data into train and test
from sklearn.model_selection import train_test_split

from sagemaker.sklearn.estimator import SKLearn


# Define S3 bucket

# SageMaker session
sagemaker_session = sagemaker.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


# import input data from S3

In [3]:

bucket = 'mysagemakerprojects'

data_key = 'cancer-prediction/test_data.csv'

data_location = f's3://{bucket}/{data_key}'

cancer_data = pd.read_csv(data_location)

cancer_data.head(10)

,age,gender,cancer_type,cancer_stage,outcome
0,27,male,colon,4,survived
1,78,female,breast,1,survived
2,42,female,breast,3,survived
3,38,female,breast,4,died
4,46,female,breast,2,survived
5,73,female,breast,2,survived
6,11,male,lung,3,survived
7,60,male,colon,2,died
8,100,female,colon,1,survived
9,42,male,lung,4,died


In [4]:
cancer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   age           1000 non-null   int64 
 1   gender        1000 non-null   object
 2   cancer_type   1000 non-null   object
 3   cancer_stage  1000 non-null   int64 
 4   outcome       1000 non-null   object
dtypes: int64(2), object(3)
memory usage: 39.2+ KB


In [5]:
cancer_data.isna().sum()

age             0
gender          0
cancer_type     0
cancer_stage    0
outcome         0
dtype: int64

In [6]:
print(cancer_data['outcome'].unique())

['survived' 'died']


In [7]:
cancer_data.head()

,age,gender,cancer_type,cancer_stage,outcome
0,27,male,colon,4,survived
1,78,female,breast,1,survived
2,42,female,breast,3,survived
3,38,female,breast,4,died
4,46,female,breast,2,survived


In [8]:

def preprocessing_data(cancer_data):
    data = cancer_data.copy()
    data['outcome'] = data['outcome'].map({'died': 0, 'survived': 1})
    # Create the OneHotEncoder
    encoder = OneHotEncoder()
    
    for i in ['gender', 'cancer_type']:
        # Fit and transform the data
        encoded_data = encoder.fit_transform(data[[i]])
        
        # Create a new DataFrame from the encoded data
        encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out([i]))
        
        # Join the encoded DataFrame with the original DataFrame
        data = data.join(encoded_df)
        
    data = data.drop(['cancer_type', 'gender'], axis=1)
    
    X = data.drop('outcome', axis=1)  # Features
    y = data['outcome']  # Target
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # Upload train/test data back to S3
    train_data = pd.concat([X_train, y_train], axis=1)
    test_data = pd.concat([X_test, y_test], axis=1)

    return train_data, test_data
    



In [9]:
train_data, test_data = preprocessing_data(cancer_data)

In [10]:
def loading_to_s3(sagemaker_session, train_data, test_data):
    train_data.to_csv('train_data.csv', index=False)
    test_data.to_csv('test_data.csv', index=False)
    
    # Upload the data to S3 for training
    train_s3_uri = sagemaker_session.upload_data('train_data.csv', bucket=bucket, key_prefix=f'{prefix}/train')
    print('laoded ', train_s3_uri)
    test_s3_uri = sagemaker_session.upload_data('test_data.csv', bucket=bucket, key_prefix=f'{prefix}/test')
    print('laoded ', test_s3_uri)

    return train_s3_uri, test_s3_uri


train_s3_uri, test_s3_uri = loading_to_s3(sagemaker_session, train_data, test_data)


laoded  s3://mysagemakerprojects/cancer-prediction/train/train_data.csv
laoded  s3://mysagemakerprojects/cancer-prediction/test/test_data.csv


In [11]:
! ls -l

total 76
-rw-r--r-- 1 sagemaker-user users 15598 Oct  3 17:38 cancer_prediction.ipynb
-rw-r--r-- 1 sagemaker-user users  3493 Oct  2 17:43 test.csv
-rw-r--r-- 1 sagemaker-user users  6318 Oct  3 17:38 test_data.csv
-rw-r--r-- 1 sagemaker-user users 14108 Oct  2 17:43 train.csv
-rw-r--r-- 1 sagemaker-user users  1406 Oct  3 17:35 train.py
-rw-r--r-- 1 sagemaker-user users 24904 Oct  3 17:38 train_data.csv


In [12]:
train_data.head()

,age,cancer_stage,gender_female,gender_male,cancer_type_breast,cancer_type_colon,cancer_type_lung,cancer_type_prostate,outcome
473,79,2,1.0,0.0,0.0,1.0,0.0,0.0,1
494,37,3,0.0,1.0,0.0,1.0,0.0,0.0,0
832,10,3,0.0,1.0,0.0,0.0,0.0,1.0,1
0,27,4,0.0,1.0,0.0,1.0,0.0,0.0,1
650,17,2,1.0,0.0,0.0,1.0,0.0,0.0,1


In [13]:
%%writefile train.py
import argparse
import os
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib

def model_fn(model_dir):
    """Load the model for inference"""
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    return model

if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    # SageMaker specific arguments
    parser.add_argument('--output-data-dir', type=str, default=os.getenv('SM_OUTPUT_DATA_DIR'))
    parser.add_argument('--model-dir', type=str, default=os.getenv('SM_MODEL_DIR'))
    parser.add_argument('--train-dir', type=str, default=os.getenv('SM_CHANNEL_TRAIN'))

    args = parser.parse_args()

    # Load the training data
    train_data = pd.read_csv(os.path.join(args.train_dir, 'train_data.csv'))
    
    # Separate the target from the features
    X = train_data.drop('outcome', axis=1)
    y = train_data['outcome']
    
    # Train the Linear Regression model
    model = LinearRegression()
    model.fit(X, y)

    # Save the model to the model directory so SageMaker can upload it to S3
    joblib.dump(model, os.path.join(args.model_dir, "model.joblib"))
    
    # Test the model on the training data
    y_pred = model.predict(X)
    mse = mean_squared_error(y, y_pred)
    print(f'Mean Squared Error on Training Data: {mse}')


Overwriting train.py


In [14]:
script_path = 'train.py'

In [15]:
train_data.head()

,age,cancer_stage,gender_female,gender_male,cancer_type_breast,cancer_type_colon,cancer_type_lung,cancer_type_prostate,outcome
473,79,2,1.0,0.0,0.0,1.0,0.0,0.0,1
494,37,3,0.0,1.0,0.0,1.0,0.0,0.0,0
832,10,3,0.0,1.0,0.0,0.0,0.0,1.0,1
0,27,4,0.0,1.0,0.0,1.0,0.0,0.0,1
650,17,2,1.0,0.0,0.0,1.0,0.0,0.0,1


In [16]:
train_s3_uri

's3://mysagemakerprojects/cancer-prediction/train/train_data.csv'

In [17]:

# Define the SKLearn estimator
sklearn = SKLearn(
    entry_point=script_path,               # path to the training script ('train.py')
    role='sagemakeruser',                  # IAM role with permissions
    instance_count=1,                      # number of instances
    instance_type='ml.m5.large',           # type of instance
    framework_version='0.23-1',            # scikit-learn version
    py_version='py3',                      # Python version
    output_path=f's3://{bucket}/{prefix}/output',  # output path for model artifacts
    sagemaker_session=sagemaker.Session()  # session to manage interactions with SageMaker
)

# Train the model
sklearn.fit({'train': train_s3_uri})


INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2024-10-03-17-39-00-369


2024-10-03 17:39:00 Starting - Starting the training job......
2024-10-03 17:39:40 Starting - Preparing the instances for training...
2024-10-03 17:40:12 Downloading - Downloading input data...
2024-10-03 17:40:37 Downloading - Downloading the training image...
2024-10-03 17:41:28 Training - Training image download completed. Training in progress.
2024-10-03 17:41:28 Uploading - Uploading generated training model2024-10-03 17:41:22,808 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-10-03 17:41:22,812 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-10-03 17:41:22,856 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-10-03 17:41:23,068 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-10-03 17:41:23,080 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-10-03 17:41:23,093 sagemaker-training-toolkit I

In [18]:
from sagemaker.model import ModelPackage

# Register the trained model
model_package_group_name = 'CancerPredictionModelPackageGroup'

# Register the model in the SageMaker Model Registry
model_package = sklearn.register(
    content_types=["text/csv"],                       # input content type
    response_types=["text/csv"],                      # output content type
    inference_instances=["ml.m5.large"],              # instance for inference
    transform_instances=["ml.m5.large"],              # instance for batch transform jobs
    model_package_group_name=model_package_group_name # model package group name for versioning
)

print(f"Model package ARN: {model_package.model_package_arn}")


INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-390403890405


Model package ARN: arn:aws:sagemaker:us-east-1:390403890405:model-package/CancerPredictionModelPackageGroup/1


In [19]:
# Deploy the trained model to an endpoint
sklearn_predictor = sklearn.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large'  # instance type for endpoint
)

INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2024-10-03-17-42-19-234
INFO:sagemaker:Creating endpoint-config with name sagemaker-scikit-learn-2024-10-03-17-42-19-234
INFO:sagemaker:Creating endpoint with name sagemaker-scikit-learn-2024-10-03-17-42-19-234


------!

In [20]:
import boto3

# Initialize a SageMaker client
sagemaker_client = boto3.client('sagemaker')

# List all the endpoints in your account
response = sagemaker_client.list_endpoints()

# Print the list of endpoints
for endpoint in response['Endpoints']:
    print(f"Endpoint Name: {endpoint['EndpointName']}, Status: {endpoint['EndpointStatus']}")


Endpoint Name: sagemaker-scikit-learn-2024-10-03-17-42-19-234, Status: InService
